In [47]:
%reload_ext autoreload
%autoreload 2

import os
import ast
import json
import importlib 
import itertools
import collections
import pprint
import pandas as pd
import pymongo

import climatools.atm.absorbers as absorbers
import climatools.lblnew.bestfit_params as bestfits
from climatools.lblnew import setup_bestfit, setup_overlap
import climatools.lblnew.pipeline as pipe_lblnew
import climatools.cliradlw.setup as setup_cliradlw
import climatools.cliradlw.pipeline as pipe_cliradlw
import climatools.cliradlw.runrecord as runrecord
import climatools.cliradlw.utils as utils_cliradlw


from IPython import display

In [48]:
pd.set_option('max_colwidth', -1)

In [49]:
client = pymongo.MongoClient('localhost', 27017)

In [50]:
ATMPROS = [('mls', 294), ('saw', 257), ('trp', 300)]

In [51]:
COMMITNUMBER = '1013f91' #'523cbb7'

In [52]:
def make_query(param):
    return {f'param.{n}': v for n, v in param.items()}

def get_nonoverlap_params():
    return [{'commitnumber': COMMITNUMBER,
             'molecule': {molecule: conc},
             'band': [band],
             'atmpro': atmpro, 'tsfc': tsfc}
            for band, molconc in absorbers.nongreys_byband().items()
            for molecule, conc in molconc.items()
            for atmpro, tsfc in ATMPROS]

def get_overlap_params():
    return [{'commitnumber': COMMITNUMBER,
             'molecule': molconc,
             'band': [band],
             'atmpro': atmpro, 'tsfc': tsfc}
            for band, molconc in absorbers.nongreys_byband().items() if len(molconc) > 1
            for atmpro, tsfc in ATMPROS]
 
def cnt_docs(param=None):
    return client.cliradnew.lw.count(make_query(param))

def notebook_exists(param=None):
    d = pipe_cliradlw.get_analysis_dir(param=param, setup=setup_cliradlw)
    try:
        return 'results_cliradlw.ipynb' in os.listdir(d)
    except FileNotFoundError:
        return False

In [53]:
def all_mls_params():
    '''
    All clirad-lw input parameter dictionaries the
    results for which will be displayed in the weblink
    table.
    '''
    params = get_nonoverlap_params() + get_overlap_params()
    params = [p for p in params if p['atmpro'] == 'mls']
    return params

In [54]:
def get_hrefs_clirad(param=None):
    hrefs = []
    for atmpro, _ in ATMPROS:
        p = param.copy()
        p['atmpro'] = atmpro
        if notebook_exists(param=p):
            url = pipe_cliradlw.nbviewer_url(param=p, 
                                             setup=setup_cliradlw)
            s = f'''<a href="{url}">{atmpro}</a>'''
            hrefs.append(s)
    s_hrefs = ' '.join(hrefs)
    return s_hrefs

def tb_hrefs_clirad():
    df = pd.DataFrame()   
    for param in all_mls_params():
        hrefs = get_hrefs_clirad(param=param)
        bands = [str(b) for b in param['band']]
        df.loc['+'.join(param['molecule']), 
               '+'.join(bands)] = hrefs
        
    df = df.fillna('')
    
    bmap = utils_cliradlw.mapband_new2old()
    lblnew_bands = [bmap[int(b)] for b in df.columns]
    columns = pd.MultiIndex.from_arrays([lblnew_bands, df.columns],
                                        names=['lblnew', 'clirad'])
    df.columns = columns
    return df

In [55]:
def script():
    df = tb_hrefs_clirad()
    
    s_html = df.to_html(escape=False)
    display.display(display.HTML(s_html))

In [56]:
script()

lblnew,1,2,3a,3b,3c,4,5,6,7,8,9
clirad,1,2,3,4,5,6,7,8,9,10,11
h2o,mls saw trp,mls saw trp,mls saw trp,mls saw trp,mls saw trp,mls saw trp,mls saw trp,mls saw trp,mls saw trp,mls saw trp,mls saw trp
co2,,,mls saw trp,mls saw trp,mls saw trp,mls saw trp,mls saw trp,,,,mls saw trp
n2o,,,mls saw trp,,,,,,mls saw trp,,
o3,,,,,,,mls saw trp,,,,
ch4,,,,,,,,,mls saw trp,,
co2+h2o+n2o,,,mls saw trp,,,,,,,,
co2+h2o,,,,,,,,,,,
co2+h2o+o3,,,,,,,,,,,
ch4+h2o+n2o,,,,,,,,,,,


In [57]:
display.HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')